# update-date-added

This script takes the 'date-added' field from a bibtex file, searches the entry with the same name in a Zotero database, and updates the 'date-added' in the Zotero database with the one from the bibtex. It hence assumes that the Zotero database contains the entries from the bibtex.

Why would you want to do that?

When you migrate from another reference manager to Zotero. Zotero does not import the date-added field. Hence, you would want to run this script after you've imported all references.

In [ ]:
import datetime
import sqlite3

import bibtexparser
from bibtexparser.bparser import BibTexParser
from bibtexparser.customization import convert_to_unicode

## File Defintions

In [ ]:
ZOTERO_SQL_FILE = '/<path-to-file>'
BIBTEX_FILE = '/<path-to-file>'

## Functions

In [ ]:
BIBTEX_REPLACEMENTS = {
    "{\\textquoteleft}": "‘",
    "{\\textquoteright}": "’",
    "{\\textemdash}": "—",
    "?!": "⁈",
    "{\\textendash}": "–",
    "\u2009{\textdegree}": " °",
    "{&}": "&",
    "ß": "\s s"
}


def update_entry(entry_title, new_date_added, conn):
    """Updates 'date-added' of Zotero entry.
    
    Parameters:
        * entry_title: the title of the entry
        * new_date_added: the date-added to replace the existing one
        * conn: a connection to the Zotero sqlite database
        
    Raises:
        * Valueerror if entry cannot be found or title is ambigious.
    """
    cursor = conn.execute('SELECT items.dateAdded, items.key \
                          FROM itemData \
                          INNER JOIN items ON items.itemID=itemData.itemID \
                          INNER JOIN itemDataValues ON itemDataValues.valueID=itemData.valueID \
                          INNER JOIN fields ON itemData.fieldID=fields.fieldID \
                          where itemDataValues.value="{}" and fields.fieldName="title";'.format(entry_title))
    all_entries = list(cursor)
    if len(all_entries) == 0:
        msg = "Did not find entry with title {} added on {}. Skipping.".format(entry_title, new_date_added)
        raise ValueError(msg)
    elif len(all_entries) > 1:
        msg = "Found several entries with title {}. Skipping.".format(entry_title)
        raise ValueError(msg)
    entry = all_entries[0]
    key = entry[1]
    conn.execute('UPDATE items \
                  SET dateAdded="{}" \
                  WHERE items.key="{}";'.format(new_date_added.strftime("%Y-%m-%d %T"), key))
        

def entries_and_their_date_added(path_to_bibtex):
    """Extracts title and date added from bibtex file."""
    with open(path_to_bibtex) as bibtex_file:
        bibtex_str = bibtex_file.read()

    parser = BibTexParser()
    parser.customization = convert_to_unicode
    bib_database = bibtexparser.loads(bibtex_str, parser=parser)
    for entry in bib_database.entries:
        title = entry["title"]
        for original, replacement in BIBTEX_REPLACEMENTS.items():
            title = title.replace(original, replacement)
        new_date_added = datetime.datetime.strptime(entry["date-added"], "%Y-%m-%dT%H:%M:%S%Z") 
        yield (title, new_date_added)

## Execution

In [ ]:
response = input("Updating '{}' based on '{}'.\nAre you sure? Is Zotero closed? [y/n].".format(ZOTERO_SQL_FILE, BIBTEX_FILE))
if response != "y":
    print("Aborted.")
with sqlite3.connect(ZOTERO_SQL_FILE) as conn:
    for entry, new_date_added in entries_and_their_date_added(BIBTEX_FILE):
        try:
            update_entry(entry_title=entry, new_date_added=new_date_added, conn=conn)
        except ValueError as ve:
            print(ve)